In [69]:
from universal import UniversalDS, ChrData
from topologicalFeatures import Cliques, Bases, BasesOfBases


In [70]:
# # A = UniversalDS("C:\\Users\\asizo\\Documents\\myPrograms\\dataFiles\\data-pvalue-0.7-fin.json")
# # B = UniversalDS("C:\\Users\\asizo\\Documents\\myPrograms\\dataFiles\\data-pvalue-5-fin.json")
# C = UniversalDS("C:\\Users\\asizo\\Documents\\myPrograms\\dataFiles\\data-pvalue-10-fin.json")
# # A.DS = "Tissue pcHi-C"
# # B.DS = "Blood cell pcHi-C"
# C.DS = "Tissue Hi-C"

In [71]:
# ch = "chr6"
# # aChData = ChrData(A, ch)
# # bChData = ChrData(B, ch)
# cChData = ChrData(C, ch)

In [72]:
import matplotlib.pyplot as plt

In [73]:
class General():
    #Creates all data for matplotlib to plot on one plot
    def __init__(self, chData):
        self.chData = chData
    def getLinkPoints(self, links):
        return [self.chData.segmentIndToMidpoint[el[0]] for el in links] +\
                [self.chData.segmentIndToMidpoint[el[1]] for el in links]
    def getLinks(self):
        self.setLinks()
        return self.linkPoints

In [74]:
class LinkEndpoints(General):
    def __init__(self, chData):
        super().__init__(chData)
    def setLinks(self): #sets linkEndpoints
        self.linkPoints = self.getLinkPoints(self.chData.allLinks)


In [75]:
class C3Endpoints(General):
    def __init__(self, chData, minC3TissueCount=1):
        super().__init__(chData)
        self.minC3TissueCount = minC3TissueCount
    def setLinks(self): #sets C3Endpoints self.linkPoints       
        A3 = Cliques(self.chData, self.minC3TissueCount)
        self.linkPoints = self.getLinkPoints(A3.getLinksList())
        

In [76]:
from numpy import log2

In [77]:

class SkEndpoints(General):
    def __init__(self, chData):
        super().__init__(chData)
    def setLinks(self): #sets SkEndpoints self.linkPoints       
        bob = BasesOfBases(self.chData)
        baseDeg = int(log2(len(bob.links)))
        self.linkPoints = self.getLinkPoints(bob.reduce(baseDeg))

In [78]:
class Drawer():
    def __init__(self, template):
        self.template = template
        self.draftContainer = []
        for v in self.template["instructions"]:
            U = UniversalDS(v["fn"])
            self.chData = ChrData(U, ch=self.template["ch"])
            if v["type"] == "Link endpoints":
                LL = LinkEndpoints(self.chData)
            elif v["type"] == "Triangle endpoints":
                LL = C3Endpoints(self.chData, minC3TissueCount=v["minC3TissueCount"])
            elif v["type"] == "Support endpoints":
                LL = SkEndpoints(self.chData)
            else:
                raise Exception("invalid type in instructions")
            
            links = LL.getLinks()
            self.draftContainer.append({"links": links, "params": v})
        if self.template["JSONfn"] is not None:
            import json
            with open(self.template["JSONfn"], 'w') as f:
                DD = {"template": self.template,
                      "draftContainer": self.draftContainer, }
                json.dump(DD, f)
            
        self.drawLinks()
    
    def drawLinks(self):
        #uses self.draftContainer
        fig, ax = plt.subplots()
        ax.set(xlabel=self.template["xlabel"])
        ax.set(ylabel=self.template["ylabel"])
        fig.suptitle(self.template["title"], size=18)
        
        bins = 250
        for draft in self.draftContainer:
            params = draft["params"]
            ax.hist(x=draft["links"], bins=bins, histtype='step', label=params["label"], fill=True, alpha=params["alpha"], density=True,
                     facecolor=params["facecolor"], hatch=params["hatch"], linewidth=0.1, edgecolor="black")
        
        

        ax.set_xlim(left=0)
        ax.ticklabel_format(axis='x', scilimits=(6,6) )
        ax.tick_params(labelsize=16)
        ax.legend(loc="upper right")
        fig.set_size_inches(16.,10.)
        plt.savefig('IMG-{title}.png'.format(title=self.template["title"]+"png"), dpi=600)
        plt.savefig('IMG-{title}.svg'.format(title=self.template["title"]+"svg"))
        #plt.show()
        plt.clf()
        

In [79]:
template = {
    "title": "Different objects of chr6 of Tissue Hi-C",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of link endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Link endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Link endpoints",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Triangle endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "C3 endpoints",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Support endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "S(k) endpoints",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"


In [80]:
D = Drawer(template)

Bases 18 of bases calculated. 12952 bases found


<Figure size 1600x1000 with 0 Axes>

In [81]:
template = {
    "title": "Different objects of chr6 of Tissue pcHi-C",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of link endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Link endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Link endpoints",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Triangle endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "C3 endpoints",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Support endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "S(k) endpoints",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"

In [82]:
D = Drawer(template)

Bases 17 of bases calculated. 4771 bases found


<Figure size 1600x1000 with 0 Axes>

In [83]:
template = {
    "title": "Different objects of chr6 of Blood cell pcHi-C",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of link endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Link endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Link endpoints",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Triangle endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "C3 endpoints",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Support endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "S(k) endpoints",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"

In [84]:
D = Drawer(template)

Bases 15 of bases calculated. 3098 bases found


<Figure size 1600x1000 with 0 Axes>

In [85]:
template = {
    "title": "Link loci in different datasets",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of link endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Link endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Blood cell pcHi-C",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Link endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue pcHi-C",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Link endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue Hi-C",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"

In [86]:
D = Drawer(template)

<Figure size 1600x1000 with 0 Axes>

In [87]:
template = {
    "title": "C3 loci in different datasets",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of C3 endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Triangle endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Blood cell pcHi-C",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Triangle endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue pcHi-C",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Triangle endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue Hi-C",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"

In [88]:
D = Drawer(template)

<Figure size 1600x1000 with 0 Axes>

In [89]:
template = {
    "title": "S(k) loci in different datasets",
    "xlabel": 'Loci in bp',
    "ylabel": 'Normalized count of S(k) endpoints',
    "ch": "chr6",
    "JSONfn": "fileWithDrawableData.json", #or None
    "binSize": 25000*2,
    "instructions": [
        {
            "fn" : "../sampleData/data-pvalue-5-fin-min.json",
            "type": "Support endpoints", # "Triangle endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Blood cell pcHi-C",
            "facecolor": "red", 
            "hatch": '-----',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-0.7-fin-min.json",
            "type": "Support endpoints", # "Link endpoints" or "Support endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue pcHi-C",
            "facecolor": "yellow", 
            "hatch": '///////',
            "alpha": 0.3,
        },
        {
            "fn" : "../sampleData/data-pvalue-10-fin-min.json",
            "type": "Support endpoints", # "Link endpoints" or "Support endpoints" or "Triangle endpoints"
            "minC3TissueCount": 1,
            "label": "Tissue Hi-C",
            "facecolor": "black", 
            "hatch": '\\\\\\\\\\\\',
            "alpha": 0.3,
        }
    ],  
}
template["JSONfn"] = template["title"]+".json"

In [90]:
D = Drawer(template)

Bases 15 of bases calculated. 3098 bases found
Bases 17 of bases calculated. 4771 bases found
Bases 18 of bases calculated. 12952 bases found


<Figure size 1600x1000 with 0 Axes>